In [301]:
%matplotlib inline
import matplotlib.pyplot as plt
init_notebook_mode(connected=True)
import pandas as pd
import numpy as np
import re 
import os 

In [302]:
filePath = os.path.join('data/DataAnalyst.csv')

In [303]:
df = pd.read_csv(filePath)

In [304]:
df.head(2)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1


In [305]:
# removed spaces and lower cased all column names to make them uniform
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [306]:
# removed unnecessary column 
df = df.drop('unnamed:_0',axis=1)

In [307]:
# inspect data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          2253 non-null   object 
 1   salary_estimate    2253 non-null   object 
 2   job_description    2253 non-null   object 
 3   rating             2253 non-null   float64
 4   company_name       2252 non-null   object 
 5   location           2253 non-null   object 
 6   headquarters       2253 non-null   object 
 7   size               2253 non-null   object 
 8   founded            2253 non-null   int64  
 9   type_of_ownership  2253 non-null   object 
 10  industry           2253 non-null   object 
 11  sector             2253 non-null   object 
 12  revenue            2253 non-null   object 
 13  competitors        2253 non-null   object 
 14  easy_apply         2253 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 264.1+ KB


In [308]:
df['lower'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[0]),dtype='float64'))
df['upper'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[1]),dtype='float64'))

In [309]:
df['city'] = df['location'].apply(lambda x: x.split(',')[0].strip())
df['state'] = df['location'].apply(lambda x: x.split(',')[1].strip())
df['state'] = df['state'].replace('Arapahoe','CO')

In [310]:
df['industry'].value_counts()

-1                                          353
IT Services                                 325
Staffing & Outsourcing                      323
Health Care Services & Hospitals            151
Computer Hardware & Software                111
                                           ... 
Sporting Goods Stores                         1
News Outlet                                   1
Consumer Electronics & Appliances Stores      1
Consumer Product Rental                       1
Catering & Food Service Contractors           1
Name: industry, Length: 89, dtype: int64

In [311]:
# obtain the average salary expecation
df['avg_salary(K)'] =  df['upper'].add(df['lower']).div(2)

In [312]:
groupSalary = df.groupby('state', as_index=False).agg({'avg_salary(K)':['mean','max','min','count']})
groupSalary.columns = groupSalary.columns.map(' '.join).str.strip().str.replace(' ','_')
groupSalary['avg_salary(K)_mean'] = round(groupSalary['avg_salary(K)_mean'],1)
salaryData = groupSalary.sort_values(by='avg_salary(K)_mean', ascending=False).reset_index(drop=True)
salaryData

,state,avg_salary(K)_mean,avg_salary(K)_max,avg_salary(K)_min,avg_salary(K)_count
0,CA,88.4,150.0,40.0,626
1,IL,78.3,113.0,59.0,164
2,CO,73.5,91.0,62.0,96
3,NJ,73.0,106.0,39.5,86
4,NY,71.4,106.0,39.5,345
5,AZ,70.8,74.0,61.5,97
6,SC,68.3,76.0,64.5,3
7,NC,68.1,76.0,57.0,90
8,VA,65.2,80.5,56.0,48
9,WA,64.8,78.0,51.0,53


In [313]:
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [327]:
mapData = dict(
    type='choropleth',
    locations = salaryData['state'].to_list(),
    locationmode = 'USA-states',
    z = salaryData['avg_salary(K)_mean'].to_list(),
    colorscale = 'Blues',
    colorbar = {'title': 'Avg Salary (1000s)'}
)

In [328]:
layout = dict(title = 'Data Analyst USA Average Salary', 
                geo = dict(scope = 'usa',
                            showlakes = True,
                            lakecolor = 'rgb(85,173,240)'))

In [329]:
choromap = go.Figure(data = [mapData], layout = layout)

In [330]:
choromap.show()